# Feature Selection  with the $\chi^2$ test

When handling classification with large numbers of potential categorical explanatory variables (for example text) we would need to find a method to **select** the most relevant variables.
 
In possible technique is the $\chi^2$ test were we search for variables that are the least independent of  class labels. 

## Preliminaries

### Imports

In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, chi2

import sys
sys.path.append("../..")
from E4525_ML import plots
from E4525_ML.text import stem_tokenizer

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

### Data Directories

In [2]:
raw_data_dir=r"../../raw/C50/C50train"
test_dir    =r"../../raw/C50/C50test"
data_dir=r"../../data/C50"


## Document Data

### Corpus (List of documents)

In [3]:
documents_filename=data_dir+"/C50_documents.csv"
documents=pd.read_csv(documents_filename,index_col="document_id")
Y=documents["label"]
documents.head()

,filename,label
document_id,,
0,../../raw/C50/C50train/ScottHillis/253868newsM...,ScottHillis
1,../../raw/C50/C50train/ScottHillis/305692newsM...,ScottHillis
2,../../raw/C50/C50train/ScottHillis/340736newsM...,ScottHillis
3,../../raw/C50/C50train/ScottHillis/140340newsM...,ScottHillis
4,../../raw/C50/C50train/ScottHillis/126593newsM...,ScottHillis


In [4]:
test_documents_filename=data_dir+"/C50_test_documents.csv"
test_documents=pd.read_csv(test_documents_filename,index_col="document_id")
Y_test=test_documents["label"]
test_documents.head()

,filename,label
document_id,,
0,../../raw/C50/C50test/ScottHillis/373999newsML...,ScottHillis
1,../../raw/C50/C50test/ScottHillis/348602newsML...,ScottHillis
2,../../raw/C50/C50test/ScottHillis/387913newsML...,ScottHillis
3,../../raw/C50/C50test/ScottHillis/392527newsML...,ScottHillis
4,../../raw/C50/C50test/ScottHillis/417664newsML...,ScottHillis


### Load Pre-Trained Features

#### Set Features

In [5]:
set_vectorizer_filename=   data_dir+"/set_vectorizer.p"
set_vectorizer=pickle.load(open(set_vectorizer_filename,"rb"))

In [6]:
# get dimension to word mapping
index_2_word=set_vectorizer.get_feature_names()

In [7]:
set_features_filename=data_dir+"/"+"set_features.p"
set_features=pickle.load(open(set_features_filename,"rb"))

In [8]:
set_test_features_filename=data_dir+"/"+"set_test_features.p"
set_test_features=pickle.load(open(set_test_features_filename,"rb"))

## Selecting words relevant for Classification 

Naive Bayes assumes uses each word in corpus independently to build a classifier.

We would like to build a classifier that, somehow,  takes into account **interactions between words**. 

The difficulty is that the vocabulary size $\approx 30k$ is too large to model all possible interactions. 

We need a procedure to **select** only a few relevant **features** (words).

The idea is that we will select words are are **least independently distributed** over all the possible classes.

We are using text as an example here, but the same ideas apply in pretty much the same way to any classification problem with **categorical** variables.

For numerical values, a $R^2$ test plays the same role 

### $\chi^2$ Test of Independence

With represent a document  $i$ as the vector of word counts $X_{i,d}$.  Where the index $i$ runs documents $i=1,\dots N$, and  $d$ runs over words (the corpus vocabulary) $d=1,\dots V$.


In [9]:
X=set_features
N=X.shape[0]
V=X.shape[1]
print("N",N)
print("V",X.shape[1])

N 2500
V 28060


Document classes are one hot-encoded $Z_{i,k}$ , for $k=1,\cdots K$, where $K$ is the number of classes.


In [10]:
dummies=pd.get_dummies(Y,prefix="",prefix_sep="",sparse=True)
labels=dummies.columns
Z=dummies.to_coo() # Most entries are zero, so we want Z represented as a sparse matrix
             # if you remove this line, calculations become very slow
print(Z.shape)

(2500, 50)


The frequency with which documents are assigned to class $k$ is
$$
        f_k = \frac{1}{N}\sum_{i=1}^N Z_{i,k}
$$


In [11]:
f=Z.sum(axis=0)/N
f.shape

(1, 50)

The number of times that word $d$ appears in documents of class $k$ is 

$$
    n_{k,d} = \sum_i Z_{i,k} X_{i,d} = Z^T X.
$$


In [12]:
word_counts=Z.T.dot(X)
word_counts.shape

(50, 28060)

The total times a word appears in the corpus is 

$$
    c_{d} = \sum_k n_{k,d}
$$


In [13]:
corpus_counts=word_counts.sum(axis=0)
corpus_counts.shape

(1, 28060)

If word counts are **independently distributed** over the document classes $k$ we would expect a per class  word count of
$$
    e_{k,d} = f_k \,c_{d}
$$

In [14]:
expected=(f.T*corpus_counts)
expected=np.asarray(expected) # this object is a "matrix" but we want a regular array
expected.shape

(50, 28060)


If word $d$ was distributed independently over the classes $k$
the following quantity
$$
   C^2_d= \sum_k \frac{\left(n_{k,d} - e_{k,d}\right)^2}{e_{k,d}}
$$
   
would be distributed as $C^2_d \sim \chi^2_{K-1}$, a  $\chi^2$ variate with $K-1$ degrees of freedom.



In [15]:
# Difference between actual and expected word counts
df=(word_counts - expected)
df=np.asarray(df) # again we don't want matrix but array objects
df.shape

(50, 28060)

In [16]:
# Normalized square difference 
df2=df*df/expected
df2.shape

(50, 28060)

In [17]:
# Sum of squares of normalized gaussians
C2=df2.sum(axis=0)
C2.shape

(28060,)

### Feature Selection using $\chi^2$ test

We  simply pick the the 
$F$ words that are the **least independently distributed** as measured by the $\chi^2$ statistic.



In [18]:
F=300

In [19]:
best_idx=C2.argsort()[-F:] # pick largest F score indexes
best_X=X[:,best_idx] # pick F best features for each document
best_X.shape

(2500, 300)

Let's look  at the 10 most significant words in the corpus

In [20]:
# Let's see the most significant words
print("word","index","C2")
for idx in best_idx[-10:]:
    print(index_2_word[idx],idx,C2[idx])

word index C2
cocoa 9729 1862.4999999999993
pragu 21054 1869.2881355932175
moscow 18598 1885.3333333333344
automak 7263 1915.0439560439534
toronto 25827 1944.1908396946528
5017 4322 2008.9999999999975
abidjan 6022 2008.9999999999986
colombia 9803 2134.9999999999973
czech 10642 2300.324503311257
tel+44 25302 2352.0000000000014


In [21]:
best_indexes=reversed(list(best_idx[-10:]))

In [22]:
for idx in best_indexes:
    print(index_2_word[idx],C2[idx])
    for l,count in enumerate(word_counts[:,idx]):
        if count>0:
            print("\t",labels[l],count)

tel+44 2352.0000000000014
	 JimGilchrist   (0, 0)	48
czech 2300.324503311257
	 AlanCrosby   (0, 0)	49
	 JanLopatka   (0, 0)	50
	 JohnMastrini   (0, 0)	50
	 MureDickie   (0, 0)	1
	 TanEeLyn   (0, 0)	1
colombia 2134.9999999999973
	 JaneMacartney   (0, 0)	1
	 KarlPenhaul   (0, 0)	49
	 MatthewBunce   (0, 0)	1
	 MichaelConnor   (0, 0)	2
	 RogerFillion   (0, 0)	1
	 ScottHillis   (0, 0)	1
abidjan 2008.9999999999986
	 MatthewBunce   (0, 0)	41
5017 2008.9999999999975
	 JimGilchrist   (0, 0)	41
toronto 1944.1908396946528
	 AlanCrosby   (0, 0)	4
	 DarrenSchuettler   (0, 0)	44
	 HeatherScoffield   (0, 0)	37
	 LydiaZajc   (0, 0)	46
automak 1915.0439560439534
	 BradDorfman   (0, 0)	3
	 DavidLawder   (0, 0)	46
	 KevinDrawbaugh   (0, 0)	2
	 MichaelConnor   (0, 0)	1
	 ToddNissen   (0, 0)	39
moscow 1885.3333333333344
	 JanLopatka   (0, 0)	1
	 JaneMacartney   (0, 0)	1
	 LynnleyBrowning   (0, 0)	43
	 ScottHillis   (0, 0)	1
	 WilliamKazer   (0, 0)	2
pragu 1869.2881355932175
	 AlanCrosby   (0, 0)	37
	 JanLo

A few comments about what information we are picking up
- Jim Gilchrist seems to work on the UK (telephone country code +44). The reason we pick him up is that most of his articles leave the phone number in an **idiosyncratic** fashion `tel+44` that does not get broken up by our **tokenization** algorithm.
- Alan Crosby seems to the Reuters correspondent to the Czech republic. His articles cover Czech's stock market, politics, news, etc.. We are are picking up the articles **geographical references**, not anything about the *authors writing style*.
- JanLopakta seems to cover only the Czech's republic stock market and economics. JhonMastrini covers Czech's economic and politic news (but not the stock market). The fact that there are 3 our of 50 authors dedicated to Czech's news indicates there are **geographical** imbalances on how this dataset was collected.
- The fact that Colombia, Toronto, Moscow, Abidjan (capital of Ivory Coast, in West Africa), cocoa are other highly informative words seems to indicate different authors cover different subject matter ( a geographical region, a commodity) and that is what we **most likely learn**.

Le'ts look at their word counts

In [23]:
# Let's look at the word counts
word_counts[:,best_idx[-10:]].todense()


matrix([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 37,  0,  0,  4,  0,  0,  0, 49,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  3,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 44,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 46,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 37,  0,  0,  0,  0,  0],
        [ 0, 36,  1,  0,  0,  0,  0,  0, 50,  0],
        [ 0,  0,  1,  0,  0,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0, 41,  0,  0,  0, 48],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 45,  0,  0,  0,  0,  0,  0, 50,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],


Note that the highest score **tel+44** looks suspicious (some kind of signature).

We are learning from the data, but maybe not what we thought we were.

Most of they important words are highly concentrated, and we may be **overfitting**

Let's look at lest significant words that still meet our threshold. 
Words with ranking on the order of $F$

In [24]:
# Let's look at the least significant of the most words 300 scores
print("word","index","C2")
for idx in best_idx[:10]:
    print(index_2_word[idx],idx,C2[idx])
word_counts[:,best_idx[:10]].todense()

word index C2
paramilitari 20227 544.6923076923075
anti-drug 6800 544.6923076923075
408-8787 3805 544.6923076923076
encrypt 12026 548.6666666666674
metal 18077 549.7826086956517
leader 16761 553.2553191489361
quarantin 21563 554.2222222222226
polit 20879 556.4143302180684
daewoo 10663 557.5714285714279
foreign 13118 557.854700854701


matrix([[ 0,  0,  0, 14,  0,  0,  0,  1,  0,  9],
        [ 0,  0,  0,  0,  0,  3,  0, 11,  1, 22],
        [ 0,  0,  0,  0,  2,  2,  0,  1,  0,  3],
        [ 0,  0,  0,  0,  0, 34,  0, 28,  0, 20],
        [ 0,  0,  0,  0,  1,  4,  0,  0,  0,  5],
        [ 0,  0,  1,  0,  0,  4,  0,  2,  0,  1],
        [ 0,  0,  0,  0,  1,  4,  0,  1,  0,  9],
        [ 0,  0,  0,  0, 21, 15,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  2],
        [ 0,  0,  0,  0,  0,  7,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  1,  0,  5,  0, 16],
        [ 0,  0,  0,  0,  5,  2,  0,  6,  0, 29],
        [ 0,  0,  0,  0, 10,  3,  0,  6,  0,  2],
        [ 0,  0,  0,  0,  0, 12,  0, 13,  0, 27],
        [ 0,  0,  0,  0,  0, 23,  0, 24,  0, 27],
        [ 0,  0,  0,  0,  1,  1,  1,  1,  0,  1],
        [ 0,  0,  0,  0,  0,  6,  0, 12,  0,  0],
        [ 0,  0,  0,  0,  1,  0,  0,  2,  0,  3],
        [ 0,  0,  0,  0,  0, 13,  0, 21,  0, 32],
        [ 0,  0,  0,  0,  0,  1,  0,  2,  1,  5],


Some words seem distributed over a few authors, as we would like.
Others, however, are just only used by one author. 

A phone number inserted itself into the list.

But let's look at words with very low ranking.

In [25]:
F_least=10
# place the indexes of the F smallest C2 scores at the
# begining of partition array
partition_worst=np.argpartition(C2, F_least) 
# we are only interest on the F largest C2 scores
worst_idx = partition_worst[:F_least] 
# get the C2 scores of the best indexes.
worst_C2=C2[worst_idx]
# Not strickly necessary, but it would be nice to have
# the test scores sorted by C2 score
worst_idx=worst_idx[np.argsort(worst_C2)]


print("word","index","C2")
for idx in worst_idx[:10]:
    print(index_2_word[idx],idx,C2[idx])
word_counts[:,worst_idx[:10]].todense()

word index C2
said 22797 0.47381144238517414
ha 14193 34.33067729083664
wa 27115 37.04068117313152
thi 25502 38.90109890109892
jockey 15991 40.00000000000001
valid 26800 41.0
recipi 21913 41.0
9.4 5742 41.9999999999999
encount 12023 41.999999999999915
hell 14540 41.99999999999995


matrix([[50, 29, 32, 34,  0,  0,  1,  0,  1,  0],
        [49, 39, 47, 33,  1,  1,  1,  0,  0,  0],
        [50, 38, 50, 44,  0,  1,  0,  1,  0,  1],
        [50, 50, 50, 29,  1,  0,  0,  0,  0,  0],
        [50, 35, 47, 37,  0,  0,  1,  1,  1,  0],
        [50, 38, 44, 36,  0,  1,  0,  0,  0,  0],
        [50, 36, 38, 41,  0,  0,  1,  0,  0,  0],
        [50, 42, 39, 30,  0,  0,  0,  0,  0,  0],
        [48, 43, 40, 33,  0,  0,  0,  0,  0,  0],
        [50, 39, 41, 38,  0,  0,  0,  0,  1,  1],
        [50, 40, 41, 35,  0,  0,  0,  0,  0,  0],
        [50, 42, 44, 41,  0,  1,  1,  0,  1,  1],
        [48, 46, 47, 37,  0,  0,  0,  0,  0,  1],
        [50, 38, 45, 37,  0,  0,  0,  1,  0,  0],
        [50, 46, 48, 39,  0,  0,  0,  1,  1,  0],
        [47, 32, 29, 28,  0,  0,  0,  1,  0,  0],
        [50, 45, 49, 38,  1,  0,  0,  0,  0,  0],
        [49, 45, 35, 45,  1,  0,  0,  0,  0,  0],
        [50, 43, 43, 39,  0,  0,  0,  1,  0,  0],
        [48, 32, 44, 29,  0,  1,  0,  0,  1,  0],


Some of the words are evenly distributed, others are just very rare in the corpus

## Comparsion to Sklearn

In [26]:
sk_C2,prob=chi2(X,Z) #chi2 from sklearn.feature_selection

Our calculations agree with 'sklearn.feature_selection.chi2'

In [27]:
metrics.mean_squared_error(sk_C2,C2)

1.8609209661787304e-26

Let's select the $F$ more relevant features

Consult 'sklearn.feature_selection' for other possible choices

In [28]:
transformer=SelectKBest(chi2,F) # get the best F based on chi2 metric

In [29]:
sk_Xt=transformer.fit_transform(X,Z) # Return X with only the selected features

Let's get the indeces of the features selected by 'SelectKBest'

In [30]:
sk_indeces=transformer.get_support(True)


We will do a bit of set algebra to compare our indeces to 'skelearn''s

In [31]:
sk_idx_set=set(sk_indeces)

In [32]:
best_idx_set=set(best_idx)

In [33]:
diffs=sk_idx_set.symmetric_difference(best_idx_set)

The only difference is  two words wit the same $\chi^2$ score.
We broke the tie differently from 'sklear'

In [34]:
for idx in diffs:
    print(idx,C2[idx],sk_C2[idx])

21514 544.6923076923073 544.6923076923076
6800 544.6923076923075 544.6923076923076


### Comparison on Test data

In [35]:
X_test=set_test_features
N_test=X_test.shape[0]
V_test=X_test.shape[1]
print("N",N_test,N)
print("V",V_test,V)

N 2500 2500
V 28060 28060


In [36]:
dummies_test=pd.get_dummies(Y_test,prefix="",prefix_sep="",sparse=True)
labels_test=dummies_test.columns
Z_test=dummies_test.to_coo() # Most entries are zero, so we want Z represented as a sparse matrix
             # if you remove this line, calculations become very slow
print(Z_test.shape)

(2500, 50)


In [37]:
word_counts_test=Z_test.T.dot(X_test)
word_counts_test.shape

(50, 28060)

In [38]:
best_indexes=reversed(list(best_idx[-10:]))
for idx in best_indexes:
    print(index_2_word[idx])
    for l,counts in enumerate(zip(word_counts[:,idx],word_counts_test[:,idx])):
        count,count_test=counts
        if count>0 or count_test>0:
            print("\t",labels[l],count[0,0],count_test[0,0])

tel+44
	 JimGilchrist 48 42
czech
	 AlanCrosby 49 41
	 HeatherScoffield 0 1
	 JanLopatka 50 50
	 JohnMastrini 50 50
	 MureDickie 1 0
	 SimonCowell 0 1
	 TanEeLyn 1 0
colombia
	 JaneMacartney 1 0
	 KarlPenhaul 49 49
	 KirstinRidley 0 3
	 MatthewBunce 1 0
	 MichaelConnor 2 0
	 RogerFillion 1 0
	 ScottHillis 1 0
abidjan
	 MatthewBunce 41 40
5017
	 JimGilchrist 41 38
toronto
	 AlanCrosby 4 0
	 DarrenSchuettler 44 46
	 HeatherScoffield 37 23
	 JimGilchrist 0 5
	 KarlPenhaul 0 1
	 LydiaZajc 46 48
	 PeterHumphrey 0 1
automak
	 BradDorfman 3 0
	 DavidLawder 46 38
	 KevinDrawbaugh 2 0
	 MichaelConnor 1 0
	 RobinSidel 0 1
	 ToddNissen 39 27
moscow
	 JanLopatka 1 1
	 JaneMacartney 1 0
	 JohnMastrini 0 5
	 LynnleyBrowning 43 40
	 MarcelMichelson 0 2
	 PeterHumphrey 0 1
	 ScottHillis 1 1
	 WilliamKazer 2 0
pragu
	 AlanCrosby 37 44
	 JanLopatka 36 42
	 JohnMastrini 45 41
cocoa
	 KarlPenhaul 0 1
	 LynnleyBrowning 1 0
	 MatthewBunce 39 37
	 PatriciaCommins 0 1
	 TimFarrand 0 1


In [39]:
print(*best_indexes)